In [ ]:
!pip install gradio
import gradio as gr
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install datasets
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
md="czearing/article-title-generator"
tokenizer=AutoTokenizer.from_pretrained(md)
model=AutoModelForSeq2SeqLM.from_pretrained(md)
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.0 MB/s

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
#!pip install unrar
!unrar x /content/arxiv_data.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/arxiv_data.rar

Extracting  arxiv_data.csv                                                31% 62% 93%100%  OK 
All OK


In [ ]:
from datasets import load_dataset
#dt=pd.read_csv('arxiv_data.csv')
dt=load_dataset('/content',data_files='arxiv_data.csv')
dt
dataset = dt['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dt['train']
k=dataset.train_test_split(test_size=0.75)
del k['test']
fresh=k['train'].train_test_split(test_size=0.2)
fresh

DatasetDict({
    train: Dataset({
        features: ['titles', 'summaries', 'terms'],
        num_rows: 10354
    })
    test: Dataset({
        features: ['titles', 'summaries', 'terms'],
        num_rows: 2589
    })
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encoding = tokenizer(example_batch['summaries'] ,padding="max_length", truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encoding = tokenizer(example_batch['titles'],padding="max_length", truncation = True )


    return{
        'input_ids' : input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels': target_encoding['input_ids']
    }

In [ ]:
dt_n = fresh.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/10354 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2589 [00:00<?, ? examples/s]

In [ ]:
dt_n['train']

Dataset({
    features: ['titles', 'summaries', 'terms', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10354
})

In [ ]:
#x_train,x_test,y_train,y_test=train_test_split(input_encodings,target_encodings,test_size=0.2,random_state=0)
#train_dt=tf.data.Dataset.from_tensor_slices((dict(x_train),y_train))
sample_text=dt_n['test'][7]['summaries']
dt_n['test'][7]['titles']


'A network-based transfer learning approach to improve sales forecasting of new products'

In [ ]:
#!pip install transformers['torch']
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='result', num_train_epochs=2, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=8,
    weight_decay=0.01, logging_steps=10,
    logging_dir='log',gradient_accumulation_steps=16,save_steps=1e6
)

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                  train_dataset=dt_n['train'],
                  eval_dataset=dt_n['test']
                   )

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,17.959700
20,17.619400
30,17.087900
40,16.090700
50,14.944500
60,13.413400
70,11.644100
80,9.346100
90,6.573900
100,3.513500


In [ ]:
model.save_pretrained("Title_Generator")

In [ ]:
tokenizer.save_pretrained("Tokenizer")

('Tokenizer/tokenizer_config.json',
 'Tokenizer/special_tokens_map.json',
 'Tokenizer/tokenizer.json')

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('/content/Tokenizer')

In [ ]:
from transformers import pipeline
def generator_text(Input):
  gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":60,"min_length":10}
  #sample_text="Deep networks for Monocular Depth Estimation (MDE) have achieved promising performance recently and it is of great importance to further understand the interpretability of these networks. Existing methods attempt to provide posthoc explanations by investigating visual cues, which may not explore the internal representations learned by deep networks. In this paper, we find that some hidden units of the network are selective to certain ranges of depth, and thus such behavior can be served as a way to interpret the internal representations. Based on our observations, we quantify the interpretability of a deep MDE network by the depth selectivity of its hidden units. Moreover, we then propose a method to train interpretable MDE deep networks without changing their original architectures, by assigning a depth range for each unit to select. Experimental results demonstrate that our method is able to enhance the interpretability of deep MDE networks by largely improving the depth selectivity of their units, while not harming or even improving the depth estimation accuracy. We further provide a comprehensive analysis to show the reliability of selective units, the applicability of our method on different layers, models, and datasets, and a demonstration on analysis of model error. Source code and models are available at https://github.com/youzunzhi/InterpretableMDE ."
  title=dt_n['test'][7]['titles']
  pipe=pipeline("summarization",model="Title_Generator",tokenizer=tokenizer)
  #print('Summary:')
  #print(sample_text)
  #print('Title:')
  #print(title)
  #print("Generated Title:")
  return(pipe(sample_text,**gen_kwargs)[0]["summary_text"])

In [ ]:

!pip install -q git+https://github.com/huggingface/transformers.git
!pip install typing-extensions --upgrade

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
output_text=gr.Textbox()
gr.Interface(fn=generator_text,outputs=output_text,inputs="textbox",title="Title Generator").launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cc37dac4e244ca0830.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
